In [1]:
import sys
import random
import pickle
from pathlib import Path

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange

In [2]:
from ds.wesad.datasets import subjects_data
from ds.wesad.datasets_users import SubjectDataset, SubjectsDataset

In [3]:
numeric_derivative = True

In [8]:
ds_15_train = SubjectsDataset(
    subjects_data,
    ds_type="train",
    window_size=15,
    step=5,
    numeric_derivative=numeric_derivative,
)

In [9]:
dl_15 = DataLoader(
    ds_15_train,
    batch_size=2,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [10]:
ds_60_train = SubjectsDataset(
    subjects_data,
    ds_type="train",
    window_size=60,
    step=5,
    numeric_derivative=numeric_derivative,
)

In [11]:
dl_60 = DataLoader(
    ds_60_train,
    batch_size=2,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [12]:
in_15 = next(iter(dl_15))
in_sig_15 = in_15[0].to(torch.float32)
in_60 = next(iter(dl_60))
in_sig_60 = in_60[0].to(torch.float32)

In [13]:
in_sig_15.shape, in_sig_60.shape

(torch.Size([2, 2, 15]), torch.Size([2, 2, 60]))

In [16]:
from conv_nets.wesad_tries.models.modules.conv_x import ConvX
from conv_nets.wesad_tries.models_v2 import get_model

In [17]:
Model = get_model(60)

In [18]:
n_c_60 = Model(numeric_derivative=numeric_derivative)
n_c_60(in_sig_60)

tensor([[0.5432, 0.4568],
        [0.3106, 0.6894]], grad_fn=<SoftmaxBackward0>)

In [71]:
def init_15(self, numeric_derivative: bool = False, reduce_cls=nn.Conv1d):
    super().__init__()
    in_channels = 2 if numeric_derivative else 1
    self.seq = nn.Sequential(
        ConvX(in_channels, 4, kernel=3),
        ConvX(4, 8, kernel=3),
        ConvX(8, 16, kernel=3),
        get_reduce(reduce_cls, 8, kernel_size=2, stride=2),
        ConvX(16, 32, kernel=3),
        ConvX(32, 16, kernel=3),
        ConvX(16, 8, kernel=3),
        get_reduce(reduce_cls, 8, kernel_size=1, stride=2),
        ConvX(8, 4, kernel=3),
        ConvX(4, 2, kernel=3),
        nn.Conv1d(2, 2, kernel_size=2, stride=2),
    )
    self.softmax = nn.Softmax(dim=-1)


def forward(self, x):
    x = self.seq(x)
    x = torch.flatten(x, 1)
    return self.softmax(x)

In [72]:
def get_net(signal_len, reduce_cls: Type[nn.Module]):
    if reduce_cls not in (nn.Conv1d, nn.MaxPool1d, nn.AvgPool1d):
        raise ValueError("Unexpected type")

    init_mapping = {15: partial(init_15, reduce_cls=reduce_cls)}
    net_cls_name = f"NetUpDownCoder3_{reduce_cls.__name__[:-2]}_{signal_len}"

    return type(
        net_cls_name,
        (nn.Module,),
        {"__init__": init_mapping[signal_len], "forward": forward},
    )

In [73]:
reduce_cls = nn.Conv1d
signal_len = 15
net_cls_name = f"NetUpDownCoder3_{reduce_cls.__name__[:-2]}_{signal_len}"

In [74]:
t = type(net_cls_name, (nn.Module,), {"__init__": init_15, "forward": forward})

In [60]:
r = partial(init_15, reduce_cls=nn.Conv1d)(nn.Module())

NoneType

In [38]:
class NetUpDownCoder3_15(nn.Module):
    def __init__(self, numeric_derivative: bool = False):
        super().__init__()
        # NetUpDownCoder3_15, self
        in_channels = 2 if numeric_derivative else 1
        self.seq = torch.nn.Sequential(
            ConvX(in_channels, 4, kernel=3),
            ConvX(4, 8, kernel=3),
            nn.Conv1d(8, 8, kernel_size=1, stride=2),
            ConvX(8, 16, kernel=3),
            ConvX(16, 32, kernel=3),
            nn.Conv1d(32, 32, kernel_size=2, stride=2),
            ConvX(32, 16, kernel=3),
            ConvX(16, 8, kernel=3),
            nn.Conv1d(8, 8, kernel_size=1, stride=2),
            ConvX(8, 4, kernel=3),
            ConvX(4, 2, kernel=3),
            nn.Conv1d(2, 2, kernel_size=2, stride=2),
        )
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.seq(x)
        x = torch.flatten(x, 1)
        return self.softmax(x)

In [41]:
net = NetUpDownCoder3_15(numeric_derivative=numeric_derivative)

In [42]:
net(in_sig_15).shape

torch.Size([2, 2])

torch.Size([2, 2, 1])

In [143]:
criterion = nn.CrossEntropyLoss()

In [149]:
criterion(net(in_sig_15), in_[1])

tensor(0.7126, grad_fn=<NllLossBackward0>)

In [60]:
conv = nn.Conv1d(2, 1, kernel_size=3)

In [113]:
in_sig_15_2 = torch.reshape(
    torch.tensor(
        [1, 2, 3, 4, 2, 3, 4, 5, 0, 1, 2, 3, 1, 2, 3, 4], dtype=torch.float32
    ),
    (2, 2, 4),
)

In [114]:
conv = nn.Conv1d(2, 1, kernel_size=2, bias=False, stride=2)
conv.weight = torch.nn.Parameter(
    torch.reshape(
        torch.tensor([0, 1, 1, 2], dtype=torch.float32, requires_grad=True),
        (1, 2, 2),
    )
)
conv.weight

Parameter containing:
tensor([[[0., 1.],
         [1., 2.]]], requires_grad=True)

In [115]:
in_sig_15_2

tensor([[[1., 2., 3., 4.],
         [2., 3., 4., 5.]],

        [[0., 1., 2., 3.],
         [1., 2., 3., 4.]]])

In [151]:
def get_ds_and_dl(key="rr_intervals", numeric_derivative=False):
    ds_all_train = SubjectsDataset(
        subjects_data,
        ds_type="train",
        step=15,
        key=key,
        numeric_derivative=numeric_derivative,
    )
    ds_all_test = SubjectsDataset(
        subjects_data,
        ds_type="test",
        step=15,
        key=key,
        numeric_derivative=numeric_derivative,
    )
    return {
        "ds": {
            "train": ds_all_train,
            "test": ds_all_test,
        },
        "dl": {
            "train": DataLoader(
                ds_all_train,
                batch_size=8,
                shuffle=True,
                num_workers=1,
                pin_memory=False,
                drop_last=True,
            ),
            "test": DataLoader(
                ds_all_test,
                batch_size=1,
                shuffle=True,
                num_workers=1,
                pin_memory=False,
                drop_last=True,
            ),
        },
    }

In [153]:
ds_dl = get_ds_and_dl(key="rr_intervals", numeric_derivative=True)

In [164]:
next(iter(ds_dl.get("dl").get("test")))[0].shape

torch.Size([1, 2, 30])

In [1]:
from conv_nets.wesad_tries.models import get_model

In [4]:
v_45 = get_model(45)
v_45_inst = v_45().to("cuda")

In [11]:
v_30 = get_model(30)
v_30_inst = v_30().to("cuda")

In [7]:
import pickle

with open(
    "/home/dmo/Documents/human_func_state/human_func_state/models_dumps/wesad/steps/derivative/NetUpDownCoder3_45_ASGD_lr_1e-06/NetUpDownCoder3_45_ASGD_lr_1e-06/NetUpDownCoder3_45_ASGD_lr_1e-06_best.pkl",
    "rb",
) as f:
    v_45_pkl = pickle.load(f)

In [14]:
v_30_inst.load_state_dict(v_30_pkl.get("net_state_dict"))

NameError: name 'v_30_pkl' is not defined